In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'


In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import sys
import os

repo_root = os.path.dirname(os.getcwd())
repair_folder = os.path.join(repo_root, "humancompatible", "repair")
sys.path.insert(0, repair_folder)

from cost import *
from coupling_utils import *
from data_analysis import *
from group_blind_repair import *
from metrics import *

In [4]:
import os
path=os.path.dirname(os.getcwd())

In [5]:
class ROCpostprocess:
    def __init__(self,X_val,y_val,var_list,prediction_model,favorable_label):
        self.X_val =X_val
        self.y_val =y_val
        self.model = prediction_model
        self.positive_index = 1 # positive label
        self.var_list = var_list
        self.var_dim=len(self.var_list)
        self.ROC = self.buildROCusingval()
        self.favorable_label = favorable_label

    def buildbinarydata(self,X,y):
        df=pd.DataFrame(np.concatenate((X,y.reshape(-1,1)), axis=1),columns=self.var_list+['S','W','Y'])
        binaryLabelDataset = BinaryLabelDataset(
                            # favorable_label=self.favorable_label,
                            # unfavorable_label=0,
                            df=df[self.var_list+['S','W','Y']], #df_test.drop('X',axis=1), #[x_list+['S','W','Y']],
                            label_names=['Y'],
                            instance_weights_name=['W'],
                            protected_attribute_names=['S'],
                            privileged_protected_attributes=[np.array([1.0])],
                            unprivileged_protected_attributes=[np.array([0.])])
        return binaryLabelDataset,df

    def buildROCusingval(self):
        dataset_val = self.buildbinarydata(self.X_val,self.y_val)[0]
        dataset_val_pred = dataset_val.copy(deepcopy=True)
        dataset_val_pred.scores = self.model.predict_proba(dataset_val.features[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        privileged_groups = [{'S': 1}]
        unprivileged_groups = [{'S': 0}]
        # Metric used (should be one of allowed_metrics)
        metric_name = "Statistical parity difference"
        # Upper and lower bound on the fairness metric used
        metric_ub = 0.05
        metric_lb = -0.05
        ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                        privileged_groups=privileged_groups, 
                                        low_class_thresh=0.01, high_class_thresh=0.99,
                                        num_class_thresh=50, num_ROC_margin=10,
                                        metric_name=metric_name,
                                        metric_ub=metric_ub, metric_lb=metric_lb)
        ROC = ROC.fit(dataset_val, dataset_val_pred)
        print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
        print("Optimal ROC margin = %.4f" % ROC.ROC_margin)
        return ROC

    def postprocess(self,X_test,y_test,tv_origin): # the tv distance won't change
        dataset_test_pred,df_test = self.buildbinarydata(X_test,y_test) #.copy(deepcopy=True)
        dataset_test_pred.scores = self.model.predict_proba(X_test[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        dataset_test_pred_transf = self.ROC.predict(dataset_test_pred)
        y_pred = dataset_test_pred_transf.labels
        # return dataset_test_pred_transf.convert_to_dataframe()[0]

        di = DisparateImpact_postprocess(df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W'])
        f1_micro = f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W'])
        f1_weighted = f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W'])
        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv_origin,'method':'ROC'})
        return new_row.to_frame().T

In [ ]:
class Projpostprocess:
    
    def __init__(self,X_test,y_test,x_list,var_list,prediction_model,K,e,thresh,favorable_label=1):
        self.model = prediction_model
        self.K=K
        self.e=e
        self.x_list=x_list
        self.var_list=var_list
        self.var_dim=len(var_list)
        self.favorable_label = favorable_label

        df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1)), axis=1),columns=var_list+['S','W','Y'])
        df_test=df_test.groupby(by=var_list+['S','Y'],as_index=False).sum()
        if len(x_list)>1:
            df_test['X'] = list(zip(*[df_test[c] for c in x_list]))
            self.x_range=sorted(set(df_test['X']))
            weight=list(1/(df_test[x_list].max()-df_test[x_list].min())) # because 'education-num' range from 1 to 16 while others 1 to 4
            self.C=c_generate_higher(self.x_range,weight)
        else:
            df_test['X']=df_test[x_list]
            self.x_range=sorted(set(df_test['X']))
            self.C=c_generate(self.x_range)
        self.df_test = df_test
        self.var_range=list(pd.pivot_table(df_test,index=var_list,values=['S','W','Y']).index)
        self.distribution_generator()
        
        if thresh == 'auto':
            self.thresh_generator()
        else:
            self.thresh=thresh

    def distribution_generator(self):
        bin=len(self.x_range)
        dist=rdata_analysis(self.df_test,self.x_range,'X')
        
        dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
        
        dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
        self.px=np.matrix(dist['x']).T
        self.ptx=np.matrix(dist['t_x']).T
        if np.any(dist['x_0']==0): 
            self.p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
        else:
            self.p0=np.matrix(dist['x_0']).T 
        if np.any(dist['x_1']==0):
            self.p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
        else:
            self.p1=np.matrix(dist['x_1']).T 
        self.V=np.matrix(dist['v']).T
        self.tv_origin=sum(abs(dist['x_0']-dist['x_1']))/2
        # return px,ptx,V,p0,p1
    
    def coupling_generator(self,method,para=None):
        if method == 'unconstrained':
            coupling=baseline(self.C,self.e,self.px,self.ptx,self.K)
        elif method == 'barycentre':
            coupling=baseline(self.C,self.e,self.p0,self.p1,self.K)
        elif method == 'partial':
            coupling=partial_repair(self.C,self.e,self.px,self.ptx,self.V,para,self.K)
        return coupling

    def postprocess(self,method,para=None):
        if method == 'origin':
            y_pred=self.model.predict(np.array(self.df_test[self.var_list]))
            tv = self.tv_origin
        else:
            coupling = self.coupling_generator(method,para)
            if (method == 'unconstrained') or (method == 'partial'):
                y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
                tv=assess_tv(self.df_test,coupling,self.x_range,self.x_list,self.var_list)
                if (para != None) and (method == 'partial'):
                    method = method+'_'+str(para)
            elif method == 'barycentre':
                y_pred,tv=postprocess_bary(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
            else:
                print('Unknown method')

        di = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])
        f1_micro = f1_score(self.df_test['Y'], y_pred, average='micro',sample_weight=self.df_test['W'])
        f1_weighted = f1_score(self.df_test['Y'], y_pred, average='weighted',sample_weight=self.df_test['W'])

        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv,'method':method})
        return new_row.to_frame().T
    
    def thresh_generator(self):
        num_thresh = 10
        ba_arr = np.zeros(num_thresh)
        ba_arr1 = np.zeros(num_thresh)
        class_thresh_arr = np.linspace(0.01, 0.1, num_thresh)
        coupling=self.coupling_generator('partial',para=1e-2)
    
        for idx, thresh in enumerate(class_thresh_arr):
            y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,thresh)
            ba_arr[idx] = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
            ba_arr1[idx] = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])

        best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        best_thresh = class_thresh_arr[best_ind]
        print("Optional threshold = ",class_thresh_arr)
        print("Disparate Impact = ",ba_arr)
        print("f1 scores = ",ba_arr1)
        self.thresh = best_thresh

In [7]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [8]:
data_path='C://personal//work//repair//.venv//Lib//site-packages//aif360//data//raw//adult'

var_list=['hours-per-week','age','capital-gain','capital-loss','education-num'] #
pa='race'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.05

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [9]:
tv_dist

{'hours-per-week': np.float64(0.12216173195089294),
 'age': np.float64(0.04149230147335384),
 'capital-gain': np.float64(0.026764553949230142),
 'capital-loss': np.float64(0.014217783478743178),
 'education-num': np.float64(0.11867963282506956)}

In [10]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv(path+'/data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

NameError: name 'baseline' is not defined

In [ ]:
report

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.45499,0.672327,0.81436,0.785157,0.19857,origin
1,0.45499,0.672327,0.81436,0.785157,0.198444,unconstrained
2,0.579056,0.671156,0.809947,0.78289,0.000055,barycentre
3,0.936146,0.636093,0.710641,0.722237,0.025519,partial_0.001
4,0.992225,0.629455,0.648636,0.673547,0.19857,ROC
5,0.442332,0.679638,0.817913,0.789218,0.17519,origin
6,0.442332,0.679638,0.817913,0.789218,0.175089,unconstrained
7,0.446741,0.67798,0.813553,0.786765,0.000046,barycentre
8,0.908634,0.637094,0.711933,0.722896,0.025059,partial_0.001
9,0.886741,0.704938,0.756015,0.768854,0.17519,ROC


# Compute average feature importance

In [ ]:
importance=[]
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    importance.append(list(clf.feature_importances_))

In [ ]:
importance=np.array(importance)
print("features", var_list)
print("mean importances", importance.mean(axis=0))

features ['hours-per-week', 'age', 'capital-gain', 'capital-loss', 'education-num']
mean importances [0.09746381 0.20263391 0.3359696  0.04851905 0.31541363]
